# Demo for full text Scholar fortune teller

In [1]:
from IPython.display import Markdown, display
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from utils import (
    get_results,
    get_pdfs,
    load_data,
    split_text_to_chunks,
    get_paragraphs,
    get_titles_abstracts,
    get_embeddings,
    generate_prompt,
    generate_answer,
    get_tldr,
    create_context_chatgpt,
)
import openai
import constants
from display_utils import display_references
from tqdm.notebook import tqdm

# tqdm pandas progress
tqdm.pandas()

In [2]:
K = 10

In [3]:
# function to get the cosine similarity between the query and the text
def get_cosine_similarity(query_embedding, text_embedding):
    return cosine_similarity(query_embedding, text_embedding)[0][0]


def get_full_text_df(query):
    df = get_results(query, limit=10)
    # get the pdfs
    get_pdfs(df, "pdfs/")
    df["pdf_paths"] = df["paperId"].progress_apply(lambda x: "pdfs/" + x + ".pdf")
    # call the load_data function to get the text from the pdfs
    print("Reading pdfs...")
    df["text"] = df["pdf_paths"].progress_apply(lambda x: load_data(x))
    # break the text into paragraphs. skip nan values
    df["paragraphs"] = df["text"].apply(
        lambda x: split_text_to_chunks(x) if not pd.isna(x) else None
    )
    # get the paragraphs and titles+abstracts
    paragraphs = get_paragraphs(df)
    titles_abstracts = get_titles_abstracts(df)

    # merge the paragraphs and titles+abstracts
    all_text = paragraphs + titles_abstracts

    # make a dataframe with the paperid and the text
    df_text = pd.DataFrame(all_text, columns=["paperId", "text"])
    # get the embeddings for text

    df_text["embedding"] = df_text["text"].progress_apply(lambda x: get_embeddings(x))
    # query to get the embeddings for the query
    query_embedding = get_embeddings(query)

    # get the cosine similarity between the query and the text
    df_text["cosine_similarity"] = df_text["embedding"].apply(
        lambda x: get_cosine_similarity(query_embedding, x)
    )

    # sort the dataframe by the cosine similarity
    df_text = df_text.sort_values(by=["cosine_similarity"], ascending=False)

    # get the top 10 results
    df_text = df_text.head(K)
    # get the tldr for the top 10 results and add it to the dataframe
    print("Generating summaries...")
    df_text["tldr"] = df_text["text"].progress_apply(lambda x: get_tldr(x))
    return df_text

In [9]:
def generate_prompt(df, query):
    """Generates a prompt for the model to answer the question."""
    return answer_question_chatgpt(
        df,
        query,
        k=K,
    )


def generate_answer(prompt):
    """Generates an answer using ChatGPT."""
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are helpful research visionary, consider the future possibilities and trends in a specific research area. Analyze the current state of the field, advancements in technology, and the potential for growth and development. Offer insights into how the researcher can contribute to this evolving landscape and provide innovative ideas that address challenges or gaps in the field. Inspire the researcher to think outside the box and explore new avenues of research, while also considering ethical, social, and environmental implications. Encourage collaboration and interdisciplinary approaches to maximize the impact of their work and drive the research area towards a promising and sustainable future.",
            },
            {"role": "user", "content": prompt},
        ],
        api_key=constants.OPENAI_API_KEY,
    )
    return response.choices[0].message.content


def answer_question_chatgpt(
    df,
    question="What is the impact of creatine on cognition?",
    k=5,
    instructions="Instructions: Using the provided search results, as a machine learning and natural language scientist, brainstorm creative ideas that combine artificial intelligence and biomedical data. Consider innovative ways to analyze, interpret, and utilize large volumes of biomedical information. Explore potential applications of AI in medical diagnostics, personalized medicine, drug discovery, and disease prevention. Consider the ethical implications and challenges of integrating AI into healthcare, and imagine how advanced algorithms could revolutionize the way we understand and treat various health conditions. Develop ideas that push the boundaries of current technology and have the potential to significantly impact the future of healthcare. If you find a result relevant definitely make sure to cite the result using [[number](URL)] notation after the reference. You are free to be creative and think like a true researcher. End your answer with some innovative ideas for future directions. \nQuery:",
    max_len=3000,
    debug=False,
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    context = create_context_chatgpt(question, df, k=k)

    try:
        # Create a completions using the question and context
        # prompt = f'''{context} \n\n Instructions: Using the provided literature with sources, write a comprehensive reply to the given query. Make sure to cite results using [[number](URL)] notation after the reference. If the provided search results refer to multiple subjects with the same name, write separate answers for each subject. You can skip a citation which you dont find relevant to the query. \nQuery:{question}\nAnswer:'''
        prompt = f"""{context} \n\n{instructions} {question}\nAnswer:"""
        return prompt
    except Exception as e:
        print(e)
        return ""


def printmd(string):
    display(Markdown(string))

# the future of robotics and agriculture


In [5]:
query = "the future of robotics and agriculture"
df_text = get_full_text_df(query)
prompt = answer_question_chatgpt(
    df_text,
    question=query,
    k=K,
    instructions="Instructions: Using the provided search results, write a comprehensive reply to the given query exploring the future direction of research. If you find a result relevant make sure to cite the result using [[number](URL)] notation after the reference. For the future researchers to work on, end your answer with 5 new detailed research questions from the search results.\nQuery:",
)
print("Generating Answer...")
response = generate_answer(prompt)
printmd(response)

  0%|          | 0/10 [00:00<?, ?it/s]

Reading pdfs...


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

Generating summaries...


  0%|          | 0/10 [00:00<?, ?it/s]

Generating Answer...


The future of robotics and agriculture holds great promise for addressing the increasing global challenges of population growth, urbanization, and food security. Advancements in robotics and AI technologies have significantly impacted the agricultural sector, with a plethora of applications ranging from land preparation, sowing, planting, plant treatment, harvesting, yield estimation, and phenotyping to using specific sensors and communication technologies along with locomotion systems [[1](https://www.semanticscholar.org/paper/6760f97cd0ec4b721ceec96205a846e6b56f05bc)].

As agricultural robotics continue to evolve and tackle the pressing needs of sustainable farming practices and efficient resource utilization [[3](https://www.semanticscholar.org/paper/f2aa28506255acbdff158cdebdd04ed1bcfc73e7)], interdisciplinary approaches can be adopted for creating innovative solutions to agricultural problems. Researchers in this field are encouraged to integrate artificial intelligence, machine learning, and the Internet of Things into the core of robotics development [[4](https://www.semanticscholar.org/paper/81ebaaf7dc103d284da173003bc3a5c3beaf437d)].

Moreover, agricultural robots can play a significant role in streamlining the entire production process, from seed to harvest, while achieving remarkable results in productivity, reduced operating costs, and minimized lead times [[5](https://www.semanticscholar.org/paper/7713d97d02c9095aeee597ba5c4df1ed5b93bb5c)]. Additionally, these technologies can be used for pest control, weed mapping, micro spraying, seeding, irrigation, and harvesting to promote sustainable agriculture and food security [[6](https://www.semanticscholar.org/paper/23589e2642fa7874aba462760f0e7c1de2663b50)].

In the context of developed countries like the UK, the use of large datasets and remote sensing in combination with robotics and automation has immense potential to revolutionize the agricultural sector [[7](https://www.semanticscholar.org/paper/4f2baf4301e719a38ec1dbb86b9766ef25650c4c)]. While in developing nations like Sri Lanka, the integration of AI-enhanced robotics and automation in agriculture can significantly contribute to their economic and technological progress [[9](https://www.semanticscholar.org/paper/81ebaaf7dc103d284da173003bc3a5c3beaf437d)].

However, future researchers in agriculture and robotics should also be conscious of ethical, social, and environmental implications. The UK Engineering and Physical Sciences Research Council (EPSRC) has published a white paper on robotics and autonomous systems in agriculture to provide insight into the associated challenges and ethical considerations [[10](https://www.semanticscholar.org/paper/4f2baf4301e719a38ec1dbb86b9766ef25650c4c)].

To facilitate future innovations and breakthroughs in this research area, we propose the following detailed research questions:

1. How can AI and machine learning be integrated into agricultural robotics for improving decision-making processes in crop management, irrigation, and pest control?

2. What innovations in sensors and computer vision algorithms can enhance the capabilities of agricultural robots to detect and respond to various environmental and crop conditions?

3. How can advances in robotics facilitate the development of efficient and eco-friendly agricultural practices, considering spatial distribution, resource allocation, and waste reduction?

4. What are the socio-economic and ethical implications of robotics adoption in agriculture, and how can the benefits be maximized while addressing potential inequities in access and opportunities?

5. How can interdisciplinary collaboration contribute to overcoming existing limitations and unanticipated challenges in the development and implementation of robotic systems in agriculture?

In [6]:
query = "large language models reduce toxicity and hallucination"
df_text = get_full_text_df(query)
prompt = answer_question_chatgpt(
    df_text,
    question=query,
    k=K,
    instructions="Instructions: Using the provided search results, write a comprehensive reply to the given query exploring the future direction of research. If you find a result relevant make sure to cite the result using [[number](URL)] notation after the reference. End you answer with 5 new research questions that can guide future studies and exploration.\nQuery:",
)
print("Generating Answer...")
response = generate_answer(prompt)
printmd(response)

  0%|          | 0/10 [00:00<?, ?it/s]

Reading pdfs...


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

Generating summaries...


  0%|          | 0/9 [00:00<?, ?it/s]

Generating Answer...


The field of large language models (LLMs) has grown significantly in recent years, with advances in training techniques, model architectures, and applications. Reducing toxicity and hallucination remains a key challenge, as inappropriate or misleading content generated by LLMs can have potentially severe consequences. Recent research provides insights into how these issues can be tackled effectively.

One approach to addressing the toxicity issue in LLMs is domain-adaptive training, where self-generated datasets are used to consistently outperform existing baselines across various model sizes [[1](https://www.semanticscholar.org/paper/3f1b04aed81bfd7e5ecec9e7c23f2fe741d36171)]. This technique, combined with detoxifying LMs at scales up to 530B, shows much potential for future developments in reducing toxicity during the learning process.

When it comes to hallucination reduction, SelfCheckGPT offers a black-box approach for detection in generative LLMs [[6](https://www.semanticscholar.org/paper/34185132eff83dd475a1bbf549f4c2f3c9370876)]. By leveraging the idea of similarity and consistency in sampled responses to determine the presence of hallucinated facts, this method can assist in improving the reliability of generated content.

Collaboration and interdisciplinary approaches are essential in further advancing the field. For example, incorporating novel pruning techniques, such as Optimal BERT Surgeon (oBERT), could improve the efficiency and accuracy of LLMs [[4](https://www.semanticscholar.org/paper/6da9a81b75e7ad02867860753d1aa276673a3a77)]. Combining ideas from various areas of research may lead to innovative solutions that address the current challenges in LLMs.

Moreover, addressing ethical, social, and environmental implications should be an integral part of future research. The use of LLMs in a variety of applications, such as education [[8](https://www.semanticscholar.org/paper/524030beb0a3ccf5ae77cbcc8b9edcdf41aefb14)], calls for a deeper understanding of the potential downsides of both toxicity and hallucination in these contexts.

To drive the research area towards a sustainable and promising future, consider the following new research questions:
1. How can existing domain-adaptive training techniques be further improved to better eliminate toxicity in LLMs?
2. What are the most effective approaches to addressing hallucination across various applications, such as generating educational content or symbolic manipulation tasks?
3. Can we develop innovative post-training quantization methods to reduce the computational cost of LLMs while preserving their accuracy in detoxification or hallucination prevention?
4. How can we create more robust frameworks that tackle toxicity and hallucination in real-world contexts and applications?
5. What are the potential ethical, social, and environmental implications of reduced toxicity and hallucination in LLMs, and how can we ensure responsible innovation in the field?

In [7]:
query = "How does iron supplementation affect anemia?"
df_text = get_full_text_df(query)
prompt = answer_question_chatgpt(
    df_text,
    question=query,
    k=K,
    instructions="Instructions: Using the provided search results, write a comprehensive reply to the given query exploring the future direction of research. If you find a result relevant make sure to cite the result using [[number](URL)] notation after the reference. End you answer with 5 new research questions that can guide future studies and exploration.\nQuery:",
)
print("Generating Answer...")
response = generate_answer(prompt)
printmd(response)

  0%|          | 0/10 [00:00<?, ?it/s]

Reading pdfs...


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

Generating summaries...


  0%|          | 0/10 [00:00<?, ?it/s]

Generating Answer...


Iron supplementation has been shown to have a positive impact on anemia, with various studies supporting its effectiveness in treating and preventing the condition. Iron and vitamin C co-supplementation has been found to produce a more significant impact on hemoglobin levels than iron supplementation alone, as evidenced by a study involving female students aged 16-21 [[1](https://www.semanticscholar.org/paper/e056f91c0ba1bc3214e0f89aaf375cafc95f3da3)][[2](https://www.semanticscholar.org/paper/e056f91c0ba1bc3214e0f89aaf375cafc95f3da3)]. Supplementation in children can be guided by the criteria from the US Centers for Disease Control and Prevention (CDC) for diagnosing anemia and assessing iron nutritional status [[3](https://www.semanticscholar.org/paper/459828144e59ace72493adae1068d460d0989c46)].

In addition to iron and vitamin C co-supplementation, iron fortification has shown promise in combating anemia, including anemia related to obesity and chronic diseases [[7](https://www.semanticscholar.org/paper/d17242c18f0bf942f258db2fb163933e3ea1a8f0)]. Breastfed infants have been found to have higher iron stores, which can be built upon by iron supplementation during pregnancy, as suggested by Bothwell [[8](https://www.semanticscholar.org/paper/2cbf181eaa9bf0ab7bc4d5426da4cabf6cea53bf)][[9](https://www.semanticscholar.org/paper/2cbf181eaa9bf0ab7bc4d5426da4cabf6cea53bf)]. Furthermore, the soluble transferrin receptor (STR) has emerged as a potential biomarker for iron-deficiency anemia in piglets, which might guide future supplementation strategies [[10](https://www.semanticscholar.org/paper/459828144e59ace72493adae1068d460d0989c46)].

To drive this research area toward a promising and sustainable future, researchers should consider interdisciplinary approaches and explore new avenues that address challenges or gaps in the field. This may involve investigating innovative supplementation formulations, personalizing strategies based on individual needs, and identifying potential adverse effects. Collaboration between experts in nutrition, medicine, and public health will help maximize the impact of future research in iron supplementation and anemia.

New research questions to guide future studies and exploration:
1. How can iron supplementation strategies be personalized to better serve the diverse needs of individuals at risk of anemia?
2. Are there alternative forms or combinations of iron and other micronutrients that may lead to better absorption or effectiveness against anemia?
3. How can public health policies and programs be optimized to ensure adequate access to iron supplementation for populations most at risk of developing anemia?
4. What is the role of emerging technologies like microbiome research, genetic and epigenetic studies in understanding and addressing anemia resulting from iron deficiency?
5. Are there any novel biomarkers, akin to the soluble transferrin receptor, that can be employed to refine diagnostic and therapeutic approaches to iron-deficiency anemia?

In [8]:
query = "large language models plagerize"
df_text = get_full_text_df(query)
prompt = answer_question_chatgpt(
    df_text,
    question=query,
    k=K,
    instructions="Instructions: Using the provided search results, write a comprehensive reply to the given query exploring the future direction of research. If you find a result relevant make sure to cite the result using [[number](URL)] notation after the reference. End you answer with 5 novel research questions that can guide future studies and exploration.\nQuery:",
)
print("Generating Answer...")
response = generate_answer(prompt)
printmd(response)

  0%|          | 0/10 [00:00<?, ?it/s]

Reading pdfs...


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/77 [00:00<?, ?it/s]

Generating summaries...


  0%|          | 0/10 [00:00<?, ?it/s]

Generating Answer...


With the advancements in large language models, there has been an improvement in performance and sample efficiency on a wide range of tasks [[1](https://www.semanticscholar.org/paper/dac3a172b504f4e33c029655e9befb3386e5f63a)]. However, the concern of plagiarism has surfaced due to the ability of these models to generate text that closely resembles the training data. Codex is an example of such a model, which has been fine-tuned on publicly available code from GitHub and possesses robust Python code-writing capabilities [[8](https://www.semanticscholar.org/paper/acbdbf49f9bc3f151b93d9ca9a06009f4f6eb269)].

As the field of large language models advances, it's crucial to address issues related to plagiarism and ethical considerations. There is a need to enhance models' capability to generate original content and ensure that generated text doesn't infringe on copyright or intellectual property rights. This can be achieved through rigorous analysis of generated content and developing innovative architecture to promote originality while maintaining performance.

Collaboration is essential in overcoming plagiarism concerns, and interdisciplinary approaches can help researchers understand the problem from different angles, such as linguistics, computer science, and legal perspectives. Furthermore, continually optimizing model sizes and training tokens would contribute to augmenting the generation capabilities of the models while balancing computational requirements [[3](https://www.semanticscholar.org/paper/8342b592fe238f3d230e4959b06fd10153c45db1)].

Approaches like Low-Rank Adaptation (LoRA) can help manage trainable parameters for downstream tasks and reduce computation hardware requirements [[5](https://www.semanticscholar.org/paper/a8ca46b171467ceb2d7652fbfb67fe701ad86092)]. Researchers must carefully consider the implications of the generated content from language models and develop innovative solutions to address potential plagiarism.

To drive the research area towards a sustainable and ethically responsible future, new research opportunities should be explored, such as:

1. How can machine learning models detect and prevent plagiarism within generated content?
2. What are the most effective techniques for promoting originality in generated text without compromising the performance of large language models?
3. How can interdisciplinary collaboration help address ethical concerns related to plagiarism and copyright infringement in generated content?
4. What role can public policies and social norms play in shaping the development and deployment of large language models that avoid plagiarism?
5. How can researchers and developers ensure that language models meet ethical standards and respect intellectual property rights as they become more advanced and ubiquitous?

In [11]:
query = "future of gps tracking in marine biology"
df_text = get_full_text_df(query)
prompt = answer_question_chatgpt(
    df_text,
    question=query,
    k=K,
    instructions="Instructions: Using the provided search results, write a comprehensive reply to the given query exploring the future direction of research. If you find a result relevant make sure to cite the result using [[number](URL)] notation after the reference. End you answer with 5 novel research questions that can guide future studies and exploration.\nQuery:",
)
response = generate_answer(prompt)
printmd(query)
printmd(response)

  0%|          | 0/10 [00:00<?, ?it/s]

Reading pdfs...


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/136 [00:00<?, ?it/s]

Generating summaries...


  0%|          | 0/10 [00:00<?, ?it/s]

Generating Answer...


future of gps tracking in marine biology

GPS tracking has revolutionized our understanding of marine biology by providing detailed information on the movements and behaviors of marine animals. As technology continues to advance and become more accessible, the future of GPS tracking in marine biology is promising. Researchers can now track individual animals for extended periods of time, yielding valuable data on migration patterns, foraging habits, and breeding activities. In addition, GPS technology can be combined with other sensors such as accelerometers, gyroscopes, and video cameras to provide a more comprehensive understanding of marine animal behavior.

Currently, the use of GPS tracking is widespread, and research efforts are being focused on improving technology and data analysis techniques to improve accuracy and reduce data loss. [[2],[5]] Additionally, studies are underway to combine GPS data with environmental and climatic data to investigate how changing ocean conditions impact marine species. [[6]] Furthermore, research is needed to explore the ethical implications of GPS tracking in terms of animal welfare and how to mitigate disturbances caused by the devices. [[8]]

To contribute to these evolving research areas, future studies may consider the following research questions:

1. How can GPS tracking technology be improved to reduce battery usage and increase longevity of transmitter devices for long-term tracking studies?
2. Can GPS tracking data be combined with environmental and climatic data to better understand the impacts of ocean changes on marine animal behavior and migration patterns?
3. How can ethical concerns related to GPS tracking be addressed, particularly for endangered or threatened species?
4. How useful is GPS tracking for understanding the influence of anthropogenic activities, such as shipping lanes and fishing activities, on marine animal behavior?
5. Can GPS tracking data be used to predict population trends and identify conservation measures for marine species that are difficult to directly survey?

By addressing these important research questions, we can continue to advance our knowledge of marine biology and improve conservation efforts for these ecologically important species.

In [12]:
query = "wearable devices leverage recent findings in microbiome research"
df_text = get_full_text_df(query)
prompt = answer_question_chatgpt(
    df_text,
    question=query,
    k=K,
    instructions="Instructions: Using the provided search results, write a comprehensive reply to the given query exploring the future direction of research. If you find a result relevant make sure to cite the result using [[number](URL)] notation after the reference. End you answer with 5 novel research questions that can guide future studies and exploration.\nQuery:",
)
response = generate_answer(prompt)
printmd(query)
printmd(response)

  0%|          | 0/10 [00:00<?, ?it/s]

Reading pdfs...


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/243 [00:00<?, ?it/s]

Generating summaries...


  0%|          | 0/10 [00:00<?, ?it/s]

wearable devices leverage recent findings in microbiome research

Wearable devices have the potential to revolutionize microbiome research by enabling real-time monitoring of the microbial communities, both within the human body and in environmental settings [[3](https://www.semanticscholar.org/paper/f2e25e1c351dd6cf2e1cfaa3b5cdde316cbf5bae)]. Recent advances in sensor technology have opened up new avenues for non-invasive sample collection and assessment of exposures to environmental volatile organic compounds in real-time [[8](https://www.semanticscholar.org/paper/f2e25e1c351dd6cf2e1cfaa3b5cdde316cbf5bae)]. Additionally, machine learning and data visualization techniques are becoming increasingly important in annotating microbes and analyzing large amounts of microbiome data to provide more detailed information [[7](https://www.semanticscholar.org/paper/38c3c2a29e54400cc4fc0a758a5435d84b01f14d)]. 
Future directions in this field could include wearable devices that enable continuous monitoring of microbial populations in real-time, which can help to identify patterns and changes in microbial communities over time. Another area of research could be exploring the interaction between microbial communities and the immune system, and how wearable devices can be used to better understand the host-microbe interactions [[1](https://www.semanticscholar.org/paper/38c3c2a29e54400cc4fc0a758a5435d84b01f14d)]. The development of innovative methods for microbiome analysis, such as single-cell genomics, could enable the identification of previously uncultivable microbial taxa and assess microbial community functions [[1](https://www.semanticscholar.org/paper/38c3c2a29e54400cc4fc0a758a5435d84b01f14d)]. Research studies could include a focus on the metagenomics of the microbiome, including integrating information from the metabolome, transcriptome, and proteome [[1](https://www.semanticscholar.org/paper/38c3c2a29e54400cc4fc0a758a5435d84b01f14d)]. Lastly, the impact of personal behavior and habits, such as diet and exercise, on the human microbiome can be further explored using wearable devices that allow for precise interventions and measurements over a longer period of time [[1](https://www.semanticscholar.org/paper/38c3c2a29e54400cc4fc0a758a5435d84b01f14d)].

Novel research questions:

1. How can wearable devices be optimized for continuous, high-resolution measurement of microbiome changes in real-time?
2. How does microbial community structure and function interact with human physiology, and what are the implications for disease prevention and treatment?
3. How can single-cell genomics be applied to reveal uncultivable microbial taxa and assess microbial community functions?
4. How can metagenomic approaches integrating information from different "omes" be used to provide a more complete understanding of host-microbe interactions?
5. How do personal behaviors such as diet and exercise impact the human microbiome, and can wearable devices provide precise interventions and measurements over a longer period of time?

In [13]:
query = "future of common sense reasoning and robotic arms"
df_text = get_full_text_df(query)
prompt = answer_question_chatgpt(
    df_text,
    question=query,
    k=K,
    instructions="Instructions: Using the provided search results, write a comprehensive reply to the given query exploring the future direction of research. If you find a result relevant make sure to cite the result using [[number](URL)] notation after the reference. End you answer with 5 novel research questions that can guide future studies and exploration.\nQuery:",
)
response = generate_answer(prompt)
printmd(query)
printmd(response)

  0%|          | 0/10 [00:00<?, ?it/s]

Reading pdfs...


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

Generating summaries...


  0%|          | 0/10 [00:00<?, ?it/s]

future of common sense reasoning and robotic arms

The development of robotic arms has been progressing rapidly, and their application in various fields is becoming increasingly important. The ability to communicate with humans using pointing actions has also been studied extensively [[1](https://www.semanticscholar.org/paper/02e82381f998959bd5b1b2bd91337243782a859e)]. The crucial question that arises is how the combination of common sense reasoning in robots and their communication with humans using pointing action will shape the future. While some research has explored this, we need to further explore the following areas:

Firstly, there is a need to investigate how common sense reasoning can be programmed into robots that can use pointing actions for communicating with humans. This research can improve the overall understanding of the abilities of robotic arms, provide insights on how they can be improved and can also encourage the development of new technologies that support better interaction between humans and robots.

Secondly, future research should aim to explore the ethical and social implications of common sense reasoning and robotic arms. As the use of robots in various fields increases, it is important to consider how robots interact with humans, especially in contexts where they work collaboratively. Researchers must evaluate how humans perceive and interact with robots and determine how the use of robots affects human employment, among other social and ethical issues.

Thirdly, future research should address the impacts of a robotic arm's communication methods on their operation. For example, how do the various communication methods robots currently employ affect their performance and their ability to perform specific tasks in various contexts [[2](https://www.semanticscholar.org/paper/02e82381f998959bd5b1b2bd91337243782a859e)]?

Fourthly, future research should explore how robotic arms can develop individual strategies for their communication that optimize their performance. As the number of robotic arms that are programmed to communicate grows, there is a need to understand how different communication methods impact their ability to complete individual tasks.

Finally, there is a need to explore how the performance of robotic arms can be improved by enhancing their communication with humans by integrating common-sense reasoning. Future studies can explore how common-sense reasoning can make robotic arms more efficient and effective in accomplishing specific tasks.

In conclusion, the future of common sense reasoning and robotic arms shows many possibilities in research. The integration of common sense reasoning and communication through pointing action can improve task performance and lay the groundwork for future usage of robots in various fields. The suggested research areas can guide researchers in exploring the future potential of these technologies and their various implications. 

Novel Research Questions:

1. Can we develop a common sense reasoning system that can be integrated with robotic arms to mimic human-like communication more efficiently? 
2. How can common-sense reasoning in robots be programmed to benefit their operation, especially for their communication with humans?
3. What ethical and social implications come with integrating common-sense reasoning and robotic arms? 
4. Can we develop a framework for evaluating the performance of robotic arms based on different communication methods? 
5. How can we measure the contribution of common-sense reasoning to the overall performance of robotic arms working with humans?